__Вопросы для повторения:__

* Что такое стэк и куча?
* Что такое указатель?
* `sizeof(int *)`?
* `sizeof(int)`?
* `sizeof(uint64_t)`?
* Куда указывает `p`?

    ```c++
    std::uint64_t arr[100];
    std::uint64_t *p = &arr[0];
    p = p + 8;
    ```
    
* Где хранятся данные строк при исполнении программы?

```c++
const char* s = "hello world";

std::string s2 = "London is the capital of Great Britain";
```

# Память в С++
___

## Поговорим про представление памяти в С++

### Кеш, оперативная память, стек и куча, выделение и освобождение памяти

#### Процессор

![img](images/processor.png "Processor")

#### Арифметика указателей

```c++
// Просто хранит какой-то адрес
void* addr = 0x1000;

// Если указатель никуда не ссылается,
// надо использовать nullptr
void* invalid = nullptr;

// Размер указателя, например, 4 - это количество
// байт необходимых для размещения адреса
size_t size = sizeof(addr); // size == 4

// Теперь мы говорим компилятору как
// интерпретировать то, на что указывет
// указатель
char* charPtr = (char*) 0x1000;

// Разыменование - получение значения, находящегося 
// по указанному адресу
char c = *charPtr; // c == 1

// & - взятие адреса, теперь в charPtrPtr находится
// адрес charPtr
char** charPtrPtr = &charPtr;

int* intPtr = (int*) addr;
int i = *intPtr; // i == 0x04030201 (little endian)

int* i1 = intPtr;
int* i2 = i1 + 2;

ptrdiff_t d1 = i2 - i1; // d1 == 2

char* c1 = (char*) i1;
char* c2 = (char*) i2;

ptrdiff_t d2 = c2 - c1; // d2 == 8
```

```
T* + n -> T* + sizeof(T) * n
T* - n -> T* - sizeof(T) * n
```


### Типы памяти

#### Стек (Stack)

```c++
int i = 5;
std::string name;
char data[5];
```
> Выделение памяти на стеке очень быстрая, но стек не резиновый

#### Куча (Heap)

```c++
int* i = (int*) malloc(sizeof(int));
std::string* name = new std::string();
char* data = new char[5];
...
free(i);
delete(name);
delete[] data;
```
Память в куче выделяют new и malloc, есть сторонние менеджеры памяти.


#### valgrind - санитайзер памяти

```c++
#include <cstdlib>

int main()
{
    int* data = (int*) malloc(1024);
    return 0;
}
```

```
valgrind ./mem 
```

```
==117392== Memcheck, a memory error detector
==117392== Copyright (C) 2002-2015, and GNU GPL'd, by Julian Seward et al.
==117392== Using Valgrind-3.11.0 and LibVEX; rerun with -h for copyright info
==117392== Command: ./mem
==117392== 
==117392== 
==117392== HEAP SUMMARY:
==117392==     in use at exit: 1,024 bytes in 1 blocks
==117392==   total heap usage: 1 allocs, 0 frees, 1,024 bytes allocated
==117392== 
==117392== LEAK SUMMARY:
==117392==    definitely lost: 1,024 bytes in 1 blocks
==117392==    indirectly lost: 0 bytes in 0 blocks
==117392==      possibly lost: 0 bytes in 0 blocks
==117392==    still reachable: 0 bytes in 0 blocks
==117392==         suppressed: 0 bytes in 0 blocks
==117392== Rerun with --leak-check=full to see details of leaked memory
==117392== 
==117392== For counts of detected and suppressed errors, rerun with: -v
==117392== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)
```


Основное:
- new то же, что и malloc, только дополнительно вызывает конструкторы
- Внутри malloc есть буфер, если в буфере есть место, ваш вызов может выполниться быстро
- Если памяти в буфере нет, будет запрошена память у ОС (sbrk, VirtualAlloc) - это дорого
- ОС выделяет память страницами от 4Кб, а может быть и все 2Мб
- Стандартные аллокаторы универсальные, то есть должны быть потокобезопасны, быть одинаково эффективны для блоков разной длины, и 10 байт и 100Мб. Плата за универсальность - быстродействие


#### Глобальная память (data segment)

```c++
static const int i = 5;
static std::string name;
extern char data[5];
```
> Если не удастся разместить блок глобальной памяти, то программа даже не запустится


### Массивы

```c++
T array[maxColumns];
T value = array[x];
```

> Значение в квадратных скобках должно быть известно на этапе компиляции, увы


```c++
int data[] = { 1, 2, 3 };
int i = data[2];
```

Фактически - это вычисление смещения:

```c++
ptr = data;
ptr = ptr + 2 * sizeof(int);
i = *ptr;
```


Массив - непрерывный блок байт в памяти, sizeof(data) вернет размер массива в байтах (не элементах!). Размер массива в элементах можно вычислить: sizeof(data) / sizeof(data[0])

```c++
int* data = new int[10];
int i = data[2];
delete[] data;
```


##### Массив <-> указатель

```c++
int i[] = { 1, 2, 3 };
int* j = i;
using array = int*;
array k = (array) j;
```

### Двумерные массивы

```c++
T array[maxRows][maxColumns];
T value = array[y][x];
```

```c++
int data[][2] = { { 1, 2 },  { 3, 4 }, { 5, 6 } };
int i = data[2][1];
```

Фактически:

```c++
ptr = data;
ptr = ptr + maxColumns * sizeof(int) * 2 + 1;
i = *ptr;
```

##### Массив <-> указатель
```c++
int i[][2] = { { 1, 2 }, { 3, 4 }, { 5, 6 } };
int* j = (int*) i;
using matrix = int(*)[2];
matrix k = (matrix) j;
```

___


```c++
#include <iostream>
#include <cstdint>

int global = 0;

int main()
{
    int* heap = (int*) malloc(sizeof(int));

    std::cout << std::hex << (uint64_t) main << '\n';
    std::cout << std::hex << (uint64_t) &global << '\n';
    std::cout << std::hex << (uint64_t) heap << '\n';
    std::cout << std::hex << (uint64_t) &heap << '\n';

    // hack, how to not close terminal of your program
    char c;
    std::cin >> c;
    return 0;
}
```

In [3]:
!g++ -O0 mem.cpp -o mem --std=c++11 && ./mem

55d676648229
55d67664b27c
55d677775eb0
7ffd3c4e07e0
^C


In [ ]:
!ps ax | grep mem


##### /proc/.../maps

```
ps ax | grep mem
```

```
00400000-00401000 r-xp 00000000 08:01 2362492
        /home/mt/work/tmp/mem
00601000-00602000 r--p 00001000 08:01 2362492
        /home/mt/work/tmp/mem
00602000-00603000 rw-p 00002000 08:01 2362492
        /home/mt/work/tmp/mem
0189c000-018ce000 rw-p 00000000 00:00 0
        [heap]
7f66aaa53000-7f66aabc5000 r-xp 00000000 08:01 6826866
        /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21
7f66aadc5000-7f66aadcf000 r--p 00172000 08:01 6826866
        /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21
7f66aadcf000-7f66aadd1000 rw-p 0017c000 08:01 6826866
        /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21
7ffd55900000-7ffd55921000 rw-p 00000000 00:00 0
        [stack]
7ffd55952000-7ffd55954000 r--p 00000000 00:00 0
        [vvar]
7ffd55954000-7ffd55956000 r-xp 00000000 00:00 0
        [vdso]
ffffffffff600000-ffffffffff601000 r-xp 00000000 00:00 0
        [vsyscall]
```

#### Виртуальная память

![](images/virtual_memory.png)

- Память делится на страницы
- Страница может находится в оперативной памяти или на внешнем носителе
- Трансляция из физического адреса в виртуальный и обратно выполняется через специальные таблицы: PGD (Page Global Directory), PMD (Page Middle Directory) и PTE (Page Table Entry). В PTE хранятся физические адреса страниц
- Для ускорения трансляции адресов процессор хранит в кеше таблицу TLB (Translation lookaside buffer)
- Если обращение к памяти не может быть оттранслировано через TLB, процессор обращается к таблицам страниц и пытается загрузить PTE оттуда в TLB. Если загрузка не удалась, процессор вызывает прерывание Page Fault
- Обработчик прерывания Page Fault находится в подсистеме виртуальной памяти ядра ОС и может загрузить требуемую страницу с внешнего носителя в оперативную память


```
1 такт = 1 / частота процессора
1 / 3 GHz = 0.3 ns
                                             0.3 ns
L1 cache reference                           0.5 ns
Branch mispredict                            5   ns
```
> Неудачный if ()
```
L2 cache reference                           7   ns
Mutex lock/unlock                           25   ns
Main memory reference                      100   ns
```
> Кроме задержки (latency) есть понятие пропускной способности (throughput, bandwidth). В случае чтения из RAM - 10-50 Gb/sec
```
Compress 1K bytes with Zippy             3,000   ns
Send 1K bytes over 1 Gbps network       10,000   ns
Read 4K randomly from SSD              150,000   ns
Read 1 MB sequentially from memory     250,000   ns
Round trip within same datacenter      500,000   ns
Read 1 MB sequentially from SSD      1,000,000   ns
HDD seek                            10,000,000   ns
Read 1 MB sequentially from HDD     20,000,000   ns
Send packet CA->Netherlands->CA    150,000,000   ns
```
Источник: [https://gist.github.com/jboner/2841832](https://gist.github.com/jboner/2841832)

Иллюстрация: [https://github.com/Kobzol/hardware-effects](https://github.com/Kobzol/hardware-effects)

### Выводы из таблицы
1. Стараться укладывать данные в кеш
2. Минимизировать скачки по памяти
3. В условиях основной веткой делать ветку которая выполняется с большей вероятностью

## Вопросы?
___

# Задача
### На входе 3х мерная матрица вида:
1 2 3
4 5 6
7 8 9
### считывать значения матрицы со стандартного ввода
### задача 
* посчитать квадрат суммы всех элементов матрицы и вывести на экран с подписью
* транспонировать матрицу (может быть не сразу понятно как). Если не понятно поменять первую и третью строки местами
* вывести получившуюся матрицу во входном формате

> ### одно из решений задачив файле matrix.cpp